# Progetto Business Intelligence per i Servizi Finanziari (a.a. 2023-2024)
Luca Pinciroli 885969

## 1. Dati utilizzati

Per prima cosa importiamo le librerie che serviranno a compiere tutte le operazioni necessarie.

In [57]:
import os
import pandas as pd
import yfinance as yf
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline

Dopodiché creiamo una semplice funzione che consente di scaricare i dati giornalieri di un'azione da Yahoo Finance e salvarli in formato CSV per averli sempre a disposizione.

In [58]:
def get_stock_data(ticker, start_date, end_date):
    
    data_dir = "StockData" # Nome della cartella per i file CSV
    os.makedirs(data_dir, exist_ok=True)
    
    
    csv_file = os.path.join(data_dir, f"{ticker}.csv") # Nome del file CSV basato sul ticker
    
    # Funzione per caricare i dati dal CSV
    def load_from_csv():
        try:
            df = pd.read_csv(csv_file, index_col=0, parse_dates=True)
            df = df.loc[start_date:end_date]
            return df
        except (FileNotFoundError, pd.errors.EmptyDataError):
            return None
    
    # Funzione per scaricare i dati da Yahoo Finance e salvarli in un CSV
    def download_from_yahoo():
        df = yf.download(ticker, start=start_date, end=end_date)
        df.to_csv(csv_file)
        return df
    
    data = load_from_csv()
    
    if data is None or data.empty:
        data = download_from_yahoo()
    
    return data

Scarichiamo i dati dei titoli scelti ed inseriamoli in un DataFrame indicizzato per titolo e data.

In [59]:
start = dt.datetime(2014,5,31)
end = dt.datetime(2024,5,31)

tickers = ['AAPL', 'INTC', 'AMZN', 'WMT', 'V', 'MA']
data_frames = []

for ticker in tickers:
    df = get_stock_data(ticker, start, end)
    df['Ticker'] = ticker
    data_frames.append(df)
    
dataframe_completo = pd.concat(data_frames)
dataframe_completo.set_index(['Ticker', dataframe_completo.index], inplace=True)
dataframe_completo.sort_index(inplace=True)

dataframe_completo

Open       High        Low      Close  Adj Close  \
Ticker Date                                                                
AAPL   2014-06-02  22.641430  22.672501  22.232143  22.451786  19.859398   
       2014-06-03  22.445000  22.812143  22.437500  22.769285  20.140236   
       2014-06-04  22.765715  23.138929  22.718214  23.029285  20.370214   
       2014-06-05  23.078571  23.191786  22.950357  23.119642  20.450140   
       2014-06-06  23.210714  23.259287  23.016787  23.056070  20.393904   
...                      ...        ...        ...        ...        ...   
WMT    2024-05-23  65.169998  65.519997  64.559998  64.839996  64.839996   
       2024-05-24  65.129997  65.690002  65.010002  65.379997  65.379997   
       2024-05-28  65.220001  65.470001  64.739998  65.040001  65.040001   
       2024-05-29  64.750000  65.209999  64.650002  64.919998  64.919998   
       2024-05-30  64.930000  65.169998  64.769997  64.889999  64.889999   

                      Volume  
Ticker Date                   
AAPL   2014-06-02  369350800  
       2014-06-03  292709200  
       2014-06-04  335482000  
       2014-06-05  303805600  
       2014-06-06  349938400  
...                      ...  
WMT    2024-05-23   16222900  
       2024-05-24   11793800  
       2024-05-28   13349600  
       2024-05-29   11538600  
       2024-05-30   13053500  

[15102 rows x 6 columns]

,AAPL,AMZN,INTC,MA,V,WMT
2014-06-02 00:00:00,22.451786,15.442000,27.260000,77.050003,53.380001,25.586666
2014-06-03 00:00:00,22.769285,15.359500,27.660000,76.349998,52.830002,25.570000
2014-06-04 00:00:00,23.029285,15.339000,27.600000,76.320000,52.857498,25.709999
2014-06-05 00:00:00,23.119642,16.178499,27.660000,76.839996,53.055000,25.773333
2014-06-06 00:00:00,23.056070,16.483500,28.170000,77.470001,53.250000,25.736668
...,...,...,...,...,...,...
2024-05-23 00:00:00,186.880005,181.050003,30.080000,451.209991,274.230011,64.839996
2024-05-24 00:00:00,189.979996,180.750000,30.719999,451.179993,274.489990,65.379997
2024-05-28 00:00:00,189.990005,182.149994,31.059999,445.079987,270.980011,65.040001
2024-05-29 00:00:00,190.289993,182.020004,30.129999,442.470001,268.859985,64.919998


In [61]:
prezzi_chiusura_giornalieri.plot(figsize=(18, 10))
plt.title('Andamento dei prezzi di chiusura aggiustati dal 2014 al 2024')
plt.xlabel('Data')
plt.ylabel('Prezzo di chiusura aggiustato (USD)')
plt.legend()
plt.grid(True)
plt.show()

NameError: name 'all_stocks_prices' is not defined